In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.constraints import maxnorm

Using TensorFlow backend.


In [3]:
#ModelCreate#Applies to All grid searches used.
def create_model(input_dim=13, 
                 optimizer='adam', 
                 init_mode='uniform', 
                 activation='relu', 
                 dropout_rate=-1, 
                 weight_constraint=-1, 
                 neurons=16,
                 n_layers=2):
    # layers_n = 4 #tunable var
    # layer_width #same thing as neurons

    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=input_dim, kernel_initializer=init_mode, activation=activation))                
    for i in  range(n_layers-1): # hidden layers...
        model.add(Dense(neurons, kernel_initializer=init_mode, activation=activation))                
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [4]:
def gsBatchEpoch(input_dim=13):
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, verbose=0)
    # define the grid search parameters
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]
    param_grid = dict(batch_size=batch_size, epochs=epochs)
    return model, batch_size, epochs, param_grid

In [5]:
def gsOptimizer(input_dim=13, batch_size=10, epochs=100):
    #####GS2
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, verbose=0, batch_size=batch_size, epochs=epochs)

    # define the grid search parameters
    optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
    param_grid = dict(optimizer=optimizer)
    
    return model, optimizer, param_grid

In [6]:
def gsInitMode(input_dim=13, batch_size=10, epochs=100, optimizer='adam'):
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, optimizer=optimizer, verbose=0, batch_size=batch_size, epochs=epochs)
    init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
    param_grid = dict(init_mode=init_mode)
    return model, init_mode, param_grid

In [7]:
def gsActivation(input_dim=13, batch_size=10, epochs=100, optimizer='adam', init_mode='uniform'):
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, optimizer=optimizer, init_mode=init_mode, verbose=0, batch_size=batch_size, epochs=epochs)
    activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
    param_grid = dict(activation=activation)
    
    return model, activation, param_grid

In [8]:
def gsNeurons(input_dim=13, batch_size=10, epochs=100, optimizer='adam', init_mode='uniform', activation='relu'):
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, optimizer=optimizer, init_mode=init_mode, activation=activation, dropout_rate=dropout_rate, weight_constraint=weight_constraint, verbose=0, batch_size=batch_size, epochs=epochs)
    neurons = [1, 5, 10, 15, 20, 25, 30]
    param_grid = dict(neurons=neurons)
    return model, neurons, param_grid

In [9]:
def gsLayers(input_dim=13, batch_size=10, epochs=100, optimizer='adam', init_mode='uniform', activation='relu', neurons=16):
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, optimizer=optimizer, init_mode=init_mode, activation=activation, dropout_rate=dropout_rate, weight_constraint=weight_constraint, verbose=0, batch_size=batch_size, epochs=epochs)
    n_layers = [2, 3, 4, 5, 6, 7, 8, 9, 10]
    param_grid = dict(n_layers=n_layers)
    return model, n_layers, param_grid

In [10]:
def studentSetup():
    ####Education Data Setup
    # load dataset
    dataset = pd.read_csv("slim-xAPI-Edu-Data.csv")
    #students failed if they are in class L.
    #dataset['Failed'] = np.where(dataset['Class'] == 'L', True, False)
    dataset['gender'] = np.where(dataset['gender']=='M',1,0)
    dataset['Relation'] = np.where(dataset['Relation']=='Father',1,0)
    dataset['ParentAnsweringSurvey'] = np.where(dataset['ParentAnsweringSurvey'] == 'Yes', 1, 0)
    dataset['ParentschoolSatisfaction'] = np.where(dataset['ParentschoolSatisfaction'] == 'Yes', 1, 0)
    dataset['AbsentMoreThanWeek'] = np.where(dataset['StudentAbsenceDays'] == 'Above-7', 1, 0)
    dataset['Semester'] = np.where(dataset['Semester'] == 'F', 1, 0)
    X = dataset[['raisedhands', 'VisITedResources', 'SectionID', 'Topic', 'StageID', 'AnnouncementsView', 'Semester', 'Discussion', 'gender', 'Relation', 'ParentAnsweringSurvey', 'ParentschoolSatisfaction', 'AbsentMoreThanWeek']]
    Y = dataset[['Class']]
    
    return X, Y

In [11]:
def colourSetup():
    dataset = pd.read_csv("colour-data.csv")
    X = dataset[['R', 'G', 'B']]
    Y = dataset[['Label']]
    return X, Y

In [12]:
def weatherSetup():
    dataset = pd.read_csv("modified-monthly-data-labelled.csv")
    X = dataset.iloc[:,0:61]
    Y = dataset.iloc[:,61]
    return X, Y

In [13]:
def printResult(grid_result):
        #Printing Results / Best Parameters.
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [14]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [15]:
#####start of Student Data Analysis.
X, Y = studentSetup()

In [49]:
##### Grid Search 1 - Batch Size and Epochs.
model, batch_size, epochs, param_grid = gsBatchEpoch(input_dim=X.shape[1])

In [50]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [51]:
bestBatchSize = grid_result.best_params_.get('batch_size')
bestEpochs = grid_result.best_params_.get('epochs')
printResult(grid_result)

Best: 0.306250 using {'batch_size': 20, 'epochs': 100}
0.264583 (0.044585) with: {'batch_size': 10, 'epochs': 10}
0.264583 (0.044585) with: {'batch_size': 10, 'epochs': 50}
0.302083 (0.036917) with: {'batch_size': 10, 'epochs': 100}
0.264583 (0.044585) with: {'batch_size': 20, 'epochs': 10}
0.264583 (0.044585) with: {'batch_size': 20, 'epochs': 50}
0.306250 (0.040505) with: {'batch_size': 20, 'epochs': 100}
0.264583 (0.044585) with: {'batch_size': 40, 'epochs': 10}
0.264583 (0.044585) with: {'batch_size': 40, 'epochs': 50}
0.287500 (0.030619) with: {'batch_size': 40, 'epochs': 100}
0.264583 (0.044585) with: {'batch_size': 60, 'epochs': 10}
0.264583 (0.044585) with: {'batch_size': 60, 'epochs': 50}
0.264583 (0.044585) with: {'batch_size': 60, 'epochs': 100}
0.264583 (0.044585) with: {'batch_size': 80, 'epochs': 10}
0.264583 (0.044585) with: {'batch_size': 80, 'epochs': 50}
0.264583 (0.044585) with: {'batch_size': 80, 'epochs': 100}
0.264583 (0.044585) with: {'batch_size': 100, 'epochs':

In [52]:
##### Grid Search 2 - for best Optimizer.
model, optimizer, param_grid = gsOptimizer(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs)

In [53]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [54]:
#From GS2, we get the best optimizer.  Save this for future reference.  Pray it is not SGD.
bestOptimizer = grid_result.best_params_.get('optimizer')
printResult(grid_result)

Best: 0.322917 using {'optimizer': 'Nadam'}
0.264583 (0.044585) with: {'optimizer': 'SGD'}
0.308333 (0.042492) with: {'optimizer': 'RMSprop'}
0.293750 (0.031869) with: {'optimizer': 'Adagrad'}
0.264583 (0.044585) with: {'optimizer': 'Adadelta'}
0.312500 (0.046771) with: {'optimizer': 'Adam'}
0.264583 (0.044585) with: {'optimizer': 'Adamax'}
0.322917 (0.058704) with: {'optimizer': 'Nadam'}


In [55]:
##### Grid Search 4 - for best init_mode
model, init_mode, param_grid = gsInitMode(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer)

In [56]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [57]:
bestInitMode = grid_result.best_params_.get('init_mode')
printResult(grid_result)

Best: 0.322917 using {'init_mode': 'normal'}
0.312500 (0.046771) with: {'init_mode': 'uniform'}
0.285417 (0.030760) with: {'init_mode': 'lecun_uniform'}
0.322917 (0.058704) with: {'init_mode': 'normal'}
0.264583 (0.044585) with: {'init_mode': 'zero'}
0.264583 (0.044585) with: {'init_mode': 'glorot_normal'}
0.264583 (0.044585) with: {'init_mode': 'glorot_uniform'}
0.264583 (0.044585) with: {'init_mode': 'he_normal'}
0.295833 (0.032808) with: {'init_mode': 'he_uniform'}


In [58]:
##### Grid Search 5 - for best Activation
model, activation, param_grid = gsActivation(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode)

In [59]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [60]:
bestActivation = grid_result.best_params_.get('activation')
printResult(grid_result)

Best: 0.322917 using {'activation': 'relu'}
0.264583 (0.044585) with: {'activation': 'softmax'}
0.312500 (0.046771) with: {'activation': 'softplus'}
0.264583 (0.044585) with: {'activation': 'softsign'}
0.322917 (0.058704) with: {'activation': 'relu'}
0.266667 (0.047416) with: {'activation': 'tanh'}
0.264583 (0.044585) with: {'activation': 'sigmoid'}
0.264583 (0.044585) with: {'activation': 'hard_sigmoid'}
0.264583 (0.044585) with: {'activation': 'linear'}


In [63]:
##### Grid Search 7 - for best Neurons
model, neurons, param_grid = gsNeurons(input_dim=X.shape[1], batch_size=bestBatchSize, activation=bestActivation, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode)

In [64]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [65]:
bestNeurons = grid_result.best_params_.get('neurons')
printResult(grid_result)

Best: 0.314583 using {'neurons': 20}
0.264583 (0.044585) with: {'neurons': 1}
0.308333 (0.042492) with: {'neurons': 5}
0.312500 (0.046771) with: {'neurons': 10}
0.306250 (0.040505) with: {'neurons': 15}
0.314583 (0.049036) with: {'neurons': 20}
0.264583 (0.044585) with: {'neurons': 25}
0.310417 (0.044585) with: {'neurons': 30}


In [18]:
##### Grid Search 7 - for best Layers.
model, neurons, param_grid = gsLayers(input_dim=X.shape[1], batch_size=bestBatchSize, activation=bestActivation, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode, neurons=bestNeurons)

In [19]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [20]:
bestLayers = grid_result.best_params_.get('n_layers')
printResult(grid_result)

Best: 0.310417 using {'n_layers': 2}
0.310417 (0.044585) with: {'n_layers': 2}
0.264583 (0.044585) with: {'n_layers': 3}
0.264583 (0.044585) with: {'n_layers': 4}
0.264583 (0.044585) with: {'n_layers': 5}
0.264583 (0.044585) with: {'n_layers': 6}
0.264583 (0.044585) with: {'n_layers': 7}
0.264583 (0.044585) with: {'n_layers': 8}
0.264583 (0.044585) with: {'n_layers': 9}
0.264583 (0.044585) with: {'n_layers': 10}


In [29]:
gsKerasHP = pd.DataFrame()

In [30]:
gsKerasHP = gsKerasHP.append({'dataset':'student', 'batch_size':bestBatchSize, 'activation':bestActivation, 'epochs':bestEpochs, 'optimizer':bestOptimizer, 'init_mode':bestInitMode, 'neurons':bestNeurons, 'n_layers':bestLayers}, ignore_index=True)
gsKerasHP

,activation,batch_size,dataset,epochs,init_mode,n_layers,neurons,optimizer
0,relu,20.0,student,100.0,normal,2.0,20.0,Nadam


In [18]:
#####start of Color Data Analysis
X, Y = colourSetup()

In [19]:
##### Grid Search 1 - Batch Size and Epochs.
model, batch_size, epochs, param_grid = gsBatchEpoch(input_dim=X.shape[1])
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)


In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))


In [ ]:
bestBatchSize = grid_result.best_params_.get('batch_size')
bestEpochs = grid_result.best_params_.get('epochs')
printResult(grid_result)

In [ ]:
##### Grid Search 2 - for best Optimizer.
model, optimizer, param_grid = gsOptimizer(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
#From GS2, we get the best optimizer.  Save this for future reference.  Pray it is not SGD.
bestOptimizer = grid_result.best_params_.get('optimizer')
printResult(grid_result)

In [ ]:
##### Grid Search 4 - for best init_mode
model, init_mode, param_grid = gsInitMode(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestInitMode = grid_result.best_params_.get('init_mode')
printResult(grid_resul

In [ ]:
bestInitMode = grid_result.best_params_.get('init_mode')
printResult(grid_result)

In [ ]:
##### Grid Search 5 - for best Activation
model, activation, param_grid = gsActivation(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestActivation = grid_result.best_params_.get('activation')
printResult(grid_result)

In [ ]:
##### Grid Search 7 - for best Neurons
model, neurons, param_grid = gsNeurons(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestNeurons = grid_result.best_params_.get('neurons')
printResult(grid_result)

In [ ]:
##### Grid Search 8 - for best Layers.
model, neurons, param_grid = gsLayers(input_dim=X.shape[1], batch_size=bestBatchSize, activation=bestActivation, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode, neurons=bestNeurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)


In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestLayers = grid_result.best_params_.get('n_layers')
printResult(grid_result)

In [ ]:
gsKerasHP = gsKerasHP.append({'dataset':'color', 'batch_size':bestBatchSize, 'activation':bestActivation, 'epochs':bestEpochs, 'optimizer':bestOptimizer, 'init_mode':bestInitMode, 'neurons':bestNeurons, 'n_layers':bestLayers}, ignore_index=True)
gsKerasHP

In [ ]:
#####start of Color Data Analysis
X, Y = weatherSetup()

In [ ]:
##### Grid Search 1 - Batch Size and Epochs.
model, batch_size, epochs, param_grid = gsBatchEpoch(input_dim=X.shape[1])
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestBatchSize = grid_result.best_params_.get('batch_size')
bestEpochs = grid_result.best_params_.get('epochs')
printResult(grid_result)

In [ ]:
##### Grid Search 2 - for best Optimizer.
model, optimizer, param_grid = gsOptimizer(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
#From GS2, we get the best optimizer.  Save this for future reference.  Pray it is not SGD.
bestOptimizer = grid_result.best_params_.get('optimizer')
printResult(grid_result)

In [ ]:
##### Grid Search 4 - for best init_mode
model, init_mode, param_grid = gsInitMode(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestInitMode = grid_result.best_params_.get('init_mode')
printResult(grid_result)

In [ ]:
##### Grid Search 5 - for best Activation
model, activation, param_grid = gsActivation(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestActivation = grid_result.best_params_.get('activation')
printResult(grid_result)

In [ ]:
##### Grid Search 7 - for best Neurons
model, neurons, param_grid = gsNeurons(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestNeurons = grid_result.best_params_.get('neurons')
printResult(grid_result)

In [ ]:
##### Grid Search 8 - for best Layers.
model, neurons, param_grid = gsLayers(input_dim=X.shape[1], batch_size=bestBatchSize, activation=bestActivation, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode, neurons=bestNeurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestLayers = grid_result.best_params_.get('n_layers')
printResult(grid_result)

In [ ]:
gsKerasHP = gsKerasHP.append({'dataset':'weather', 'batch_size':bestBatchSize, 'activation':bestActivation, 'epochs':bestEpochs, 'optimizer':bestOptimizer, 'init_mode':bestInitMode, 'neurons':bestNeurons, 'n_layers':bestLayers}, ignore_index=True)
gsKerasHP